In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from shapely.geometry import shape
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt

#### Exclude tracts that are too far from they city core or have no data from any years

In [2]:
census_track_geojson = 'data/Austin_Census_Tracts_simplified.json'
score_data = 'data/austin_stats_data.csv'

exclusion_FIPS = ['48453001741', '48453001742', '48453001743', '48453001760', '48453001762',
                  '48453001764', '48453001765', 
                  '48453001766', '48453001767', 
                  '48453001768', '48453001836', '48453001837', 
                  '48453001840', '48453001841', '48453001842',
                  '48453001906', '48453001904', '48453001909', '48453001732',
                  '48453002206', '48453002303', '48453002416', '48453002417', 
                  '48453002418', '48453002205'
                 ]

with open(census_track_geojson) as f:
    geo_data = json.load(f)

#### Find representative point within each shape

In [3]:
df_geo_data = gpd.read_file(census_track_geojson)
rep_points = df_geo_data['geometry'].representative_point()
df_geo_data['rep_point_lat'] = rep_points.y
df_geo_data['rep_point_lon'] = rep_points.x

#### Only include tracts within travis county

In [4]:
df_geo_data_travis = df_geo_data[df_geo_data['FIPS'].str[:5]=='48453']
df_geo_data_travis = df_geo_data_travis[~df_geo_data_travis['FIPS'].isin(exclusion_FIPS)]

In [6]:
# Create lon_lat file
df_geo_data_travis[['FIPS','rep_point_lat','rep_point_lon']].to_csv('data/austin_lon_lat_data.csv',index=False)

In [10]:
df_geo_data_travis['FIPS']=df_geo_data_travis['FIPS'].astype(np.int64)
df_geo_data_travis = df_geo_data_travis[['FIPS','geometry']]

In [11]:
df_score = pd.read_csv(score_data)

#### Function to assign tract color based on standardized score

In [17]:
def assign_color_bin(row):
    if row[2]==-1: return 0
    elif row[2]<=0.125: return 1
    elif row[2]<=0.25: return 2
    elif row[2]<=0.375: return 3
    elif row[2]<=0.5: return 4
    elif row[2]<=0.625: return 5
    elif row[2]<=0.75: return 6
    elif row[2]<=0.875: return 7
    else: return 8    
    return -1

#### Vulnerability Score Coloring

In [29]:
scaler = MinMaxScaler()
df_score['normalized_vuln_score'] = scaler.fit_transform(df_score['Vulnerability Score'].values.reshape((-1,1)))

for i in range(5):
    austin_score_year = df_score[df_score['year']==2013+i]
    austin_score_year = pd.merge(df_geo_data_travis, austin_score_year[['FIPS','normalized_vuln_score']], on='FIPS', how='left')
    austin_score_year['normalized_vuln_score'] = austin_score_year['normalized_vuln_score'].fillna(-1)
    austin_score_year['color_bin'] = austin_score_year.apply(assign_color_bin, axis=1)
    for j in range(9):
        austin_score_year[['FIPS','normalized_vuln_score','geometry']][austin_score_year['color_bin']==j].to_file('data/Vulnerability Score/'+str(2013+i)+'/austin'+str(j)+'.geojson',driver='GeoJSON')

#### Predicted Score Coloring

In [36]:
scaler = MinMaxScaler()
df_score['normalized_pred_score'] = scaler.fit_transform(df_score['Predicted Score'].values.reshape((-1,1)))

for i in range(8):
    austin_score_year = df_score[df_score['year']==2013+i]
    austin_score_year = pd.merge(df_geo_data_travis, austin_score_year[['FIPS','normalized_pred_score']], on='FIPS', how='left')
    austin_score_year['normalized_pred_score'] = austin_score_year['normalized_pred_score'].fillna(-1)
    austin_score_year['color_bin'] = austin_score_year.apply(assign_color_bin, axis=1)
    for j in range(9):
        try:
            austin_score_year[['FIPS','normalized_pred_score','geometry']][austin_score_year['color_bin']==j].to_file('data/Predicted Score/'+str(2013+i)+'/austin'+str(j)+'.geojson',driver='GeoJSON')
        except:
            print("No tract in this group")
            pass

No tract in this group
